## ortools -cvrp

In [92]:
"""Capacited Vehicles Routing Problem (CVRP)."""
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
      [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
      [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
      [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
      [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
      [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
      [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
      [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
      [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
      [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
      [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
      [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
      [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
      [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
      [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
      [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
      [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
      [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
        # fmt: on
    ]
    data["demands"] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, ]
    data["vehicle_capacities"] = [15, 15, 15, 15]
    data["num_vehicles"] = 4
    data["depot"] = 0
    data["starts"]=[0,0,16,16]
    data["ends"]=[16,16,16,16]
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()
    display(pd.DataFrame(data["distance_matrix"]))
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["starts"],data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,548,776,696,582,274,502,194,308,194,536,502,388,354,468,776,662
1,548,0,684,308,194,502,730,354,696,742,1084,594,480,674,1016,868,1210
2,776,684,0,992,878,502,274,810,468,742,400,1278,1164,1130,788,1552,754
3,696,308,992,0,114,650,878,502,844,890,1232,514,628,822,1164,560,1358
4,582,194,878,114,0,536,764,388,730,776,1118,400,514,708,1050,674,1244
5,274,502,502,650,536,0,228,308,194,240,582,776,662,628,514,1050,708
6,502,730,274,878,764,228,0,536,194,468,354,1004,890,856,514,1278,480
7,194,354,810,502,388,308,536,0,342,388,730,468,354,320,662,742,856
8,308,696,468,844,730,194,194,342,0,274,388,810,696,662,320,1084,514
9,194,742,742,890,776,240,468,388,274,0,342,536,422,388,274,810,468


## shortests(nearest)

In [8]:
np.hstack([np.ones(1),np.zeros(1)])

array([1., 0.])

In [1]:
#import libraries
import numpy as np
import pandas as pd

distance_matrix = pd.read_csv("C:/Users/vjame/OneDrive/OneDrive - dongguk.edu/문서/카카오톡 받은 파일/final_distance.csv")
df=pd.DataFrame(distance_matrix)
dist_matrix=df.to_numpy()

def calculate_distance_matrix(df,points): #distance_matrix,수거지개수
    """
    Calculate the distance matrix between coordinates.
    """
    num_points = points
    dist_matrix =df.to_numpy()
    return dist_matrix

def calculate_total_distance(route, dist_matrix): #특정 경로= route
    """
    Calculate the total distance of a given route using the distance matrix.
    """
    total_distance = 0
    num_points = len(route)

    for i in range(num_points - 1):
        current_node = route[i]
        next_node = route[i + 1]
        total_distance += dist_matrix[current_node, next_node]

    return total_distance


def nearest_neighbor(dist_matrix, demands, capacity,num_car=5,last_node=0):
    """
    Apply the Nearest Neighbor heuristic to find initial routes for VRP.
    1.현재 가지고 있는 차량들 개수로 start_node=0 (차고지)로 설정
    2.한바퀴돌고 나서 다음부터는 매립지를 기준으로 start_node=차고지로 설정
    """
    num_points = dist_matrix.shape[0]
    visited = np.zeros(num_points-2, dtype=bool) #모두 false로 저장
    visited=np.hstack([np.ones(1,dtype=bool),visited,np.ones(1,dtype=bool)]) # 매립지는 True로 저장하기
    routes = []
    capa=[]
    num_car=5
    while (np.sum(visited) < num_points):
        if len(routes)>=num_car:
            last_node=len(df)-1 #차고지로 설정
        current_node = last_node  # Start at node 0
        current_capacity = 0
        route = [current_node]
        visited[current_node] = True #첫번째 node0 방문

        while current_capacity + demands[current_node] <= capacity:
            current = route[-1] #현재노드
            nearest = None
            min_dist = float('inf') #무한값

            for neighbor in np.where(~visited)[0]:#visit false인곳 반복
                #capa 가능한 & 최소 거리인 곳저장과정
                if demands[neighbor] + current_capacity <= capacity and dist_matrix[current, neighbor] < min_dist: 
                    nearest = neighbor #가까운 노드 저장
                    min_dist = dist_matrix[current, neighbor] #가장 가까운 거리 저장
            #남은 곳 어느곳도 capa를 담지못해서 break
            if nearest is None:
                break
            #route에 nearest 저장, visited ture변환, capa 저장
            route.append(nearest)
            visited[nearest] = True
            current_capacity += demands[nearest]
        #모든 경로들 저장
        route.append(len(df)-1)
        routes.append(route)
        capa.append(current_capacity)

    return routes,capa


def format_output(routes):
    """
    화살표로 노드 구현
    """
    num=0
    for i in range(len(routes)):
        dis=calculate_total_distance(routes[i], dist_matrix)
        num+=dis 
        print(f"Route{i}:{dis}")
        print(*routes[i],sep="->")
        print("")
    print("Objective:"+str(num))

        
#    return routes

def vrp_solver(distance_matrix,capacity):
    """
    Solve the VRP using the provided filename for coordinates and vehicle capacity.
    """
    #distance_matrix = pd.read_csv(filename)
    display(distance_matrix)
    dist_matrix=df.to_numpy()
    demands=np.ones(len(df))
    routes,capa = nearest_neighbor(dist_matrix, demands, capacity,2)
    formatted_routes = format_output(routes)
    return routes,capa


In [2]:
routes,capa=vrp_solver(df,15)

,0,1,2,3,4,5,6,7,8,9,...,162,163,164,165,166,167,168,169,170,171
0,0,1160,712,1258,1180,2213,902,1801,1636,1237,...,1452,982,1852,532,1006,959,514,1055,1347,5337
1,1160,0,546,432,690,1723,1105,1311,1146,747,...,962,492,1362,371,516,255,448,565,857,4913
2,712,546,0,624,763,1796,1178,1384,1219,820,...,1035,565,1435,248,589,132,256,638,930,4986
3,1258,432,624,0,1123,1532,1538,1120,955,556,...,771,626,1795,804,650,688,881,998,1290,5154
4,1180,690,763,1123,0,1629,1463,1213,949,1105,...,1320,1117,670,943,1141,827,1020,124,165,4221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,959,255,132,688,827,1630,1289,1448,1846,979,...,1134,836,812,459,811,0,287,1124,1416,5472
168,514,448,256,881,1020,1823,1482,1641,2039,1172,...,1327,1029,1005,346,1004,287,0,895,1187,5243
169,1055,565,638,998,124,1726,1385,1544,1551,1416,...,1230,932,908,981,907,1124,895,0,291,4347
170,1347,857,930,1290,165,1820,1835,1634,1321,1186,...,1193,1224,1200,1273,1199,1416,1187,291,0,4056


Route0:6038
0->22->101->16->69->161->66->72->55->168->43->48->167->58->18->1->171

Route1:8434
0->40->11->10->30->17->6->121->71->113->124->114->100->126->120->117->171

Route2:6818
0->165->28->15->44->26->151->59->56->64->51->169->13->4->170->20->171

Route3:7824
0->29->149->2->42->19->152->163->166->41->74->38->107->150->60->160->171

Route4:10552
0->34->75->119->159->24->37->164->32->136->9->36->73->108->162->144->171

Route5:11882
171->54->25->33->57->70->14->52->27->21->91->63->67->45->53->7->171

Route6:10673
171->109->140->8->154->143->157->156->155->153->158->142->148->147->145->146->171

Route7:11634
171->141->110->76->81->82->111->62->23->31->95->85->84->5->46->106->171

Route8:12267
171->131->96->78->89->83->99->90->93->94->88->97->80->87->86->103->171

Route9:12777
171->128->61->135->137->134->130->132->102->138->68->105->122->104->127->12->171

Route10:13415
171->125->47->35->123->39->3->49->50->118->116->115->112->98->79->77->171

Route11:17626
171->139->92->65->133->129-

In [39]:
df_2173=pd.read_csv("C:/Users/vjame/Downloads/0990_새벽_0201_0202.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 0: invalid continuation byte

In [40]:
#naver map api key
client_id = '861z66ikys';    # 본인이 할당받은 ID 입력
client_pw = 'k5M5KmlwOFtZBg027Z5q6myw7f06R1p33I1gjviv';    # 본인이 할당받은 Secret 입력

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

# 주소 목록 파일 (.csv)
data = df_2173
# 주소 목록 파일 (.xlsx)
# data = pd.read_excel('파일명.xlsx')

# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []     
for add in data['주소(지번)']:
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['addresses'] == [] :
                print("'result' not exist!")
                latitude = None
                longitude = None
            else:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
        else:
            print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([latitude, longitude])



In [41]:

np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"도로명": data['주소(지번)'].values,
                              "위도": np_geo_coordi[:, 0],
                              "경도": np_geo_coordi[:, 1]})

df_0990_xy=pd_geo_coordi
df_0990_xy.to_csv("0990_새벽")

,도로명,위도,경도
0,경기도 구리시 인창동 219-16(15m 범위),37.6046653,127.1468652
1,경기도 구리시 인창동 219-16(18m 범위),37.6046653,127.1468652
2,경기도 구리시 인창동 219-16(51m 범위),37.6046653,127.1468652
3,경기도 구리시 수택동 362-5(13m 범위),37.6004550,127.1496976
4,경기도 구리시 토평동 188-4(7m 범위),37.5946346,127.1509287
...,...,...,...
175,경기도 구리시 인창동 678-5(8m 범위),37.6038284,127.1470558
176,경기도 구리시 인창동 686(25m 범위),37.6045329,127.1451010
177,경기도 구리시 인창동 219-16(24m 범위),37.6046653,127.1468652
178,경기도 구리시 인창동 253-3(23m 범위),37.6044119,127.1472859


In [3]:
df2=pd.read_csv("C:/Users/vjame/design/final_num_171.csv")
df2=df2[["x","y"]]
df_v1=df2.iloc[routes[0]+routes[6]]
df_v2=df2.iloc[routes[1]+routes[7]]
df_v3=df2.iloc[routes[2]+routes[8]]
df_v4=df2.iloc[routes[3]+routes[9]]
df_v5=df2.iloc[routes[4]+routes[10]+routes[11]]

In [4]:
num=0
route_5=[routes[0]+routes[6],routes[1]+routes[7],routes[2]+routes[8],routes[3]+routes[9],routes[4]+routes[10]+routes[11]]
for i in range(5):
          dis=calculate_total_distance(route_5[i], dist_matrix)
          num+=dis 
          print(f"Route{i}:{dis}")
          print(*route_5[i],sep="->")
          print("")
print("Objective:"+str(num))

#0->22->101->16->69->161->66->72->55->168->43->48->167->58->18->1->171


Route0:16711
0->22->101->16->69->161->66->72->55->168->43->48->167->58->18->1->171->171->109->140->8->154->143->157->156->155->153->158->142->148->147->145->146->171

Route1:20068
0->40->11->10->30->17->6->121->71->113->124->114->100->126->120->117->171->171->141->110->76->81->82->111->62->23->31->95->85->84->5->46->106->171

Route2:19085
0->165->28->15->44->26->151->59->56->64->51->169->13->4->170->20->171->171->131->96->78->89->83->99->90->93->94->88->97->80->87->86->103->171

Route3:20601
0->29->149->2->42->19->152->163->166->41->74->38->107->150->60->160->171->171->128->61->135->137->134->130->132->102->138->68->105->122->104->127->12->171

Route4:41593
0->34->75->119->159->24->37->164->32->136->9->36->73->108->162->144->171->171->125->47->35->123->39->3->49->50->118->116->115->112->98->79->77->171->171->139->92->65->133->129->171

Objective:118058


In [5]:
# 위도, 경도
lat, lon = 126.9996417,37.56159516
# 줌 크기
zoom_size = 15

In [6]:
import folium
df2[['x', 'y']].values

array([[127.0137248 ,  37.55578326],
       [127.0151491 ,  37.56032618],
       [127.0162541 ,  37.55937801],
       [127.0181571 ,  37.56110674],
       [127.0103552 ,  37.56335459],
       [127.0234953 ,  37.56582846],
       [127.0107168 ,  37.55281785],
       [127.0178259 ,  37.56753126],
       [127.0124052 ,  37.56512224],
       [127.0163867 ,  37.5647555 ],
       [127.0124212 ,  37.55550869],
       [127.0126305 ,  37.55541568],
       [127.0187383 ,  37.56075355],
       [127.0104202 ,  37.56244413],
       [127.0108362 ,  37.56411607],
       [127.0144628 ,  37.55979865],
       [127.0137304 ,  37.55605536],
       [127.0116574 ,  37.55586007],
       [127.0148925 ,  37.56003781],
       [127.0127065 ,  37.56010798],
       [127.0104104 ,  37.5643792 ],
       [127.0165708 ,  37.56554483],
       [127.0138343 ,  37.55597948],
       [127.0209831 ,  37.56814351],
       [127.0149865 ,  37.56297117],
       [127.0069077 ,  37.56410564],
       [127.0124206 ,  37.56080691],
 

In [14]:
tile_url = 'https://tile.jawg.io/jawg-sunny/{z}/{x}/{y}{r}.png?access-token=gRLVC6bquHdBb5O59W6KhNkBFzH1rcsQ7owYnYzXZpPAkRphi9MBE0NwKUivhsPP'
# 지도 생성 (초기 위치와 줌 레벨을 설정)
map = folium.Map(location=[lon, lat], zoom_start=15)

# Jawg Streets 타일 레이어 추가
jawg_streets = folium.TileLayer(
    tiles=tile_url,
    attr='&copy; <a href="https://jawg.io" title="Tiles Courtesy of Jawg Maps" target="_blank"><b>Jawg</b>Maps</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
    min_zoom=0,
    max_zoom=22,
    name='Jawg Streets'
)
weight=5
jawg_streets.add_to(map)
folium.PolyLine(
    locations=df_v1[['y', 'x']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
    weight=weight,  # 선의 두께
    color='blue',  # 선의 색깔
    opacity=0.6  # 선의 투명도
).add_to(map)
folium.PolyLine(
    locations=df_v2[['y', 'x']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
    weight=weight,  # 선의 두께
    color='red',  # 선의 색깔
    opacity=0.6  # 선의 투명도
).add_to(map)
folium.PolyLine(
    locations=df_v3[['y', 'x']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
    weight=weight,  # 선의 두께
    color='dark',  # 선의 색깔
    opacity=0.6  # 선의 투명도
).add_to(map)
folium.PolyLine(
    locations=df_v4[['y', 'x']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
    weight=weight,  # 선의 두께
    color='orange',  # 선의 색깔
    opacity=0.6  # 선의 투명도
).add_to(map)
folium.PolyLine(
    locations=df_v5[['y', 'x']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
    weight=weight,  # 선의 두께
    color='darkblue',  # 선의 색깔
    opacity=0.6  # 선의 투명도
).add_to(map)

# 각 포인트에 마커 추가
for idx, row in df_v1.iterrows():
    folium.CircleMarker(
        location=[row['y'], row['x']],
        radius=3,  # 원의 반지름
        color='blue',  # 원의 색상
        fill=True,  # 원 내부를 채움
        fill_color='blue',  # 채움 색상
        fill_opacity=1  # 투명도
    ).add_to(map)
map

In [10]:
for i,j in df.iterrows():
          print(i,j)

0 0         0
1      1160
2       712
3      1258
4      1180
       ... 
167     959
168     514
169    1055
170    1347
171    5337
Name: 0, Length: 172, dtype: int64
1 0      1160
1         0
2       546
3       432
4       690
       ... 
167     255
168     448
169     565
170     857
171    4913
Name: 1, Length: 172, dtype: int64
2 0       712
1       546
2         0
3       624
4       763
       ... 
167     132
168     256
169     638
170     930
171    4986
Name: 2, Length: 172, dtype: int64
3 0      1258
1       432
2       624
3         0
4      1123
       ... 
167     688
168     881
169     998
170    1290
171    5154
Name: 3, Length: 172, dtype: int64
4 0      1180
1       690
2       763
3      1123
4         0
       ... 
167     827
168    1020
169     124
170     165
171    4221
Name: 4, Length: 172, dtype: int64
5 0      2213
1      1723
2      1796
3      1532
4      1629
       ... 
167    1630
168    1823
169    1726
170    1820
171    5272
Name: 5, Length: 172,

In [125]:
df_r1=pd.read_csv("C:/Users/vjame/OneDrive/OneDrive - dongguk.edu/문서/카카오톡 받은 파일/distance_2173.csv",encoding="euc-kr")
df_r2=pd.read_csv("C:/Users/vjame/OneDrive/OneDrive - dongguk.edu/문서/카카오톡 받은 파일/distance_2173(2).csv",encoding="euc-kr")
df_r3=pd.read_csv("C:/Users/vjame/OneDrive/OneDrive - dongguk.edu/문서/카카오톡 받은 파일/distance_2173(3).csv",encoding="euc-kr")

df_r4=pd.read_csv("C:/Users/vjame/OneDrive/OneDrive - dongguk.edu/문서/카카오톡 받은 파일/distance_2173(4).csv",encoding="utf-8")
df_r5=pd.read_csv("C:/Users/vjame/OneDrive/OneDrive - dongguk.edu/문서/카카오톡 받은 파일/distance_2173(5).csv",encoding="utf-8")
df_r1.columns=['Unnamed: 0', '차량번호', '보고시각', '주소(지번)', 'x', 'y']
df_r2.columns=['Unnamed: 0', '차량번호', '보고시각', '주소(지번)', 'x', 'y']
df_r3.columns=['Unnamed: 0', '차량번호', '보고시각', '주소(지번)', 'x', 'y']
df_r4.columns=['Unnamed: 0', '차량번호', '보고시각', '주소(지번)', 'x', 'y']
df_r5.columns=['Unnamed: 0', '차량번호', '보고시각', '주소(지번)', 'x', 'y']

In [68]:
df_5934_xy=df_5934_xy[["위도","경도"]].astype("float")

In [67]:
df_5934_xy

,위도,경도
0,37.567174,127.004685
1,37.567586,127.004988
2,37.567586,127.004988
3,37.567586,127.004988
4,37.567174,127.004685
...,...,...
185,37.567174,127.004685
186,37.567174,127.004685
187,37.567174,127.004685
188,37.567174,127.004685


In [83]:
tile_url = 'https://tile.jawg.io/jawg-sunny/{z}/{x}/{y}{r}.png?access-token=gRLVC6bquHdBb5O59W6KhNkBFzH1rcsQ7owYnYzXZpPAkRphi9MBE0NwKUivhsPP'
# 지도 생성 (초기 위치와 줌 레벨을 설정)
map = folium.Map(location=[lon, lat], zoom_start=15)

# Jawg Streets 타일 레이어 추가
jawg_streets = folium.TileLayer(
    tiles=tile_url,
    attr='&copy; <a href="https://jawg.io" title="Tiles Courtesy of Jawg Maps" target="_blank"><b>Jawg</b>Maps</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
    min_zoom=0,
    max_zoom=22,
    name='Jawg Streets'
)
weight=2
opacity=1
jawg_streets.add_to(map)
# folium.PolyLine(
#     locations=df_0990_xy[['위도', '경도']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
#     weight=weight,  # 선의 두께
#     color='red',  # 선의 색깔
#     opacity=opacity  # 선의 투명도
# ).add_to(map)
# folium.PolyLine(
#     locations=df_2173_xy[['위도', '경도']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
#     weight=weight,  # 선의 두께
#     color='blue',  # 선의 색깔
#     opacity=opacity  # 선의 투명도
# ).add_to(map)
# folium.PolyLine(
#     locations=df_5934_xy[['위도', '경도']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
#     weight=weight,  # 선의 두께
#     color='green',  # 선의 색깔
#     opacity=opacity  # 선의 투명도
# ).add_to(map)
folium.PolyLine(
    locations=df_7024_xy[['위도', '경도']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
    weight=weight,  # 선의 두께
    color='red',  # 선의 색깔
    opacity=opacity  # 선의 투명도
).add_to(map)
# folium.PolyLine(
#     locations=df_9917_xy[['위도', '경도']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
#     weight=weight,  # 선의 두께
#     color='purple',  # 선의 색깔
#     opacity=opacity  # 선의 투명도
# ).add_to(map)

# # 각 포인트에 마커 추가
# for idx, row in df.iterrows():
#     folium.Marker(
#         location=[row['x'], row['y']],
#         popup=f'Index: {row["index"]}',  # 팝업에 표시될 텍스트
#     ).add_to(map)
map.save("찐찐찐막기존경로red")

In [61]:
tile_url = 'https://tile.jawg.io/jawg-sunny/{z}/{x}/{y}{r}.png?access-token=gRLVC6bquHdBb5O59W6KhNkBFzH1rcsQ7owYnYzXZpPAkRphi9MBE0NwKUivhsPP'
# 지도 생성 (초기 위치와 줌 레벨을 설정)
map = folium.Map(location=[lon, lat], zoom_start=15)

# Jawg Streets 타일 레이어 추가
jawg_streets = folium.TileLayer(
    tiles=tile_url,
    attr='&copy; <a href="https://jawg.io" title="Tiles Courtesy of Jawg Maps" target="_blank"><b>Jawg</b>Maps</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
    min_zoom=0,
    max_zoom=22,
    name='Jawg Streets'
)
weight=2
opacity=1
jawg_streets.add_to(map)
folium.PolyLine(
    locations=df_r1[['y', 'x']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
    weight=weight,  # 선의 두께
    color='blue',  # 선의 색깔
    opacity=opacity  # 선의 투명도
).add_to(map)
folium.PolyLine(
    locations=df_r2[['y', 'x']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
    weight=weight,  # 선의 두께
    color='red',  # 선의 색깔
    opacity=opacity  # 선의 투명도
).add_to(map)
folium.PolyLine(
    locations=df_r3[['y', 'x']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
    weight=weight,  # 선의 두께
    color='dark',  # 선의 색깔
    opacity=opacity  # 선의 투명도
).add_to(map)
folium.PolyLine(
    locations=df_r4[['y', 'x']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
    weight=weight,  # 선의 두께
    color='orange',  # 선의 색깔
    opacity=opacity  # 선의 투명도
).add_to(map)
folium.PolyLine(
    locations=df_r5[['y', 'x']].values,  # 위도, 경도 컬럼을 이용하여 위치 리스트 생성
    weight=weight,  # 선의 두께
    color='pink',  # 선의 색깔
    opacity=opacity  # 선의 투명도
).add_to(map)

# # 각 포인트에 마커 추가
# for idx, row in df.iterrows():
#     folium.Marker(
#         location=[row['x'], row['y']],
#         popup=f'Index: {row["index"]}',  # 팝업에 표시될 텍스트
#     ).add_to(map)
map.save("ddd.html")

NameError: name 'df_r1' is not defined

In [32]:
B=np.arange(18).reshape(9,2)
def distance(B):
        dist_matrix=np.zeros((len(B),len(B)))
        for i in range(len(B)):
                for j in range(len(B)):
                        dist_matrix[i,j]=np.sum(np.abs(B[i]-B[j]))
        return dist_matrix


In [8]:
import pandas as pd
df1=pd.read_csv("C:/Users/vjame/git/design/0201새벽.csv")
df2=pd.read_csv("C:/Users/vjame/git/design/0202새벽.csv")

In [26]:
df1=df1[:21]
df2=df2[:30]

In [104]:
df1

,Unnamed: 0,차량번호,보고시각,주소(지번),x좌표,y좌표
0,1.0,800조2173,2024-02-01 00:02:02,서울특별시 중구 신당동 345-23,127.013725,37.555783
1,2.0,800조2173,2024-02-01 00:08:02,서울특별시 중구 신당동 340-101,127.012560,37.556591
2,3.0,800조2173,2024-02-01 00:14:02,서울특별시 중구 신당동 340-127,127.012447,37.555977
3,4.0,800조2173,2024-02-01 00:29:02,서울특별시 중구 신당동 370-67,127.011595,37.556143
4,5.0,800조2173,2024-02-01 00:32:02,서울특별시 중구 신당동 333-165,127.013179,37.558649
5,6.0,800조2173,2024-02-01 00:41:02,서울특별시 중구 신당동 287-62,127.015149,37.560326
6,7.0,800조2173,2024-02-01 00:50:03,서울특별시 중구 신당동 308-54,127.012665,37.559942
7,8.0,800조2173,2024-02-01 00:53:02,서울특별시 중구 신당동 236-291,127.010360,37.564125
8,9.0,800조2173,2024-02-01 00:56:03,서울특별시 중구 쌍림동 102,127.002921,37.563237
9,10.0,800조2173,2024-02-01 00:59:02,서울특별시 중구 필동2가 10-6,126.993755,37.561305


In [27]:
xy1=df1[["x좌표","y좌표"]].to_numpy()
xy2=df2[["x좌표","y좌표"]].to_numpy()
xy=np.vstack([xy1,xy2])
xy

array([[127.01372482,  37.55578326],
       [127.01256006,  37.5565914 ],
       [127.01244746,  37.55597667],
       [127.01159494,  37.55614281],
       [127.01317892,  37.55864876],
       [127.01514913,  37.56032618],
       [127.01266486,  37.55994184],
       [127.01035985,  37.56412539],
       [127.00292074,  37.56323662],
       [126.99375528,  37.56130484],
       [126.98141426,  37.5616652 ],
       [126.9685024 ,  37.56159516],
       [126.97358438,  37.56263706],
       [126.98368685,  37.56063795],
       [126.99833066,  37.56219383],
       [127.01625413,  37.55937801],
       [127.01171345,  37.55619352],
       [127.01283649,  37.55890081],
       [127.0181571 ,  37.56110674],
       [127.01539333,  37.56395095],
       [127.01536497,  37.56476095],
       [127.01254998,  37.56333302],
       [127.01185363,  37.56311605],
       [127.01270995,  37.56353294],
       [127.0117471 ,  37.56367251],
       [127.01083625,  37.56411607],
       [126.9762387 ,  37.56041189],
 

In [66]:
xy[41]

array([127.01415825,  37.5598449 ])

In [36]:
dist_matrix=distance(xy)

In [54]:
dist_matrix=np.triu(dist_matrix)

In [55]:
pd.DataFrame(dist_matrix)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,0.0,0.001973,0.001471,0.002489,0.003411,0.005967,0.005219,0.011707,0.018257,0.025491,...,0.004495,0.005355,0.005422,0.009824,0.005343,0.011910,0.045917,0.010738,0.010288,0.011130
1,0.0,0.000000,0.000727,0.001414,0.002676,0.006324,0.003455,0.009734,0.016285,0.023518,...,0.004852,0.005712,0.005779,0.010181,0.003663,0.009937,0.043944,0.008765,0.008315,0.009158
2,0.0,0.000000,0.000000,0.001019,0.003404,0.007051,0.004183,0.010236,0.016787,0.024020,...,0.005579,0.006439,0.006506,0.010908,0.004390,0.010440,0.044447,0.009267,0.008817,0.009660
3,0.0,0.000000,0.000000,0.000000,0.004090,0.007738,0.004869,0.009218,0.015768,0.023002,...,0.006265,0.007126,0.007193,0.011595,0.005077,0.009421,0.043428,0.008248,0.007799,0.008641
4,0.0,0.000000,0.000000,0.000000,0.000000,0.003648,0.001807,0.008296,0.014846,0.022080,...,0.002175,0.003036,0.003103,0.007505,0.001932,0.008499,0.042506,0.007326,0.006877,0.007719
5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.002869,0.008588,0.015139,0.022373,...,0.001472,0.000612,0.000545,0.003857,0.002661,0.008792,0.044046,0.007619,0.007169,0.008012
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006489,0.013039,0.020273,...,0.001590,0.002257,0.002324,0.006726,0.000208,0.006692,0.041177,0.005519,0.005069,0.005912
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008328,0.019425,...,0.008079,0.008496,0.008620,0.010793,0.006364,0.000304,0.043055,0.001088,0.001419,0.000630
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011097,...,0.014629,0.015047,0.015171,0.017343,0.012914,0.008632,0.034728,0.007520,0.007969,0.007698
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.021863,0.022280,0.022404,0.024577,0.020148,0.019729,0.023630,0.018337,0.018101,0.018796


In [60]:
mask = np.logical_and(dist_matrix < threshold, dist_matrix != 0)

          # 조건을 만족하는 요소 추출
filtered_values = dist_matrix[mask]
filtered_values

array([2.77696059e-04, 7.27336970e-04, 2.77252532e-04, 8.71270953e-04,
       8.13513552e-04, 4.26605869e-04, 1.69216010e-04, 5.42440569e-04,
       3.45167411e-04, 5.94480618e-04, 7.87495054e-04, 6.11733170e-04,
       5.45014591e-04, 2.07817868e-04, 4.85723592e-04, 3.04322704e-04,
       6.29580402e-04, 3.73224558e-04, 3.89526959e-04, 6.87080615e-05,
       8.38355913e-04, 3.59878765e-04, 6.62998896e-04, 6.89013313e-04,
       3.39246270e-04, 5.75262859e-04, 3.37964745e-04, 4.26359462e-04,
       6.84659867e-04, 1.75761884e-04, 2.42480463e-04, 3.50816523e-04,
       6.02121346e-04, 6.68839925e-04, 7.84813083e-04, 8.36897173e-04,
       7.62751518e-04, 7.62974256e-04, 8.60421752e-04, 1.23865747e-04,
       4.49840697e-04, 4.58826723e-04, 8.42449822e-04])

In [105]:
threshold=abs(127.01211446864-127.012630481008) +abs(37.5557735247078- 37.5554156844824)
threshold=0.0003738525934148811

In [106]:
for i in range(50):
          print()

In [107]:
mask = np.logical_and(dist_matrix < threshold, dist_matrix != 0)

# 조건을 만족하는 값들의 인덱스 찾기
indices = np.where(mask)

# 결과 출력
print("원본 행렬:")
print(dist_matrix)
print(f"값이 {threshold}보다 작고 0이 아닌 요소들의 인덱스:")
for i, j in zip(indices[0], indices[1]):
    print(f"위치: ({i}, {j}), 값: {dist_matrix[i, j]}")


원본 행렬:
[[0.         0.00197291 0.00147077 ... 0.01073783 0.01028799 0.01113044]
 [0.         0.         0.00072734 ... 0.00876492 0.00831508 0.00915753]
 [0.         0.         0.         ... 0.00926706 0.00881722 0.00965967]
 ...
 [0.         0.         0.         ... 0.         0.00044984 0.00045883]
 [0.         0.         0.         ... 0.         0.         0.00084245]
 [0.         0.         0.         ... 0.         0.         0.        ]]
값이 0.0003738525934148811보다 작고 0이 아닌 요소들의 인덱스:
위치: (0, 35), 값: 0.0002776960585109123
위치: (1, 34), 값: 0.0002772525324061803
위치: (3, 16), 값: 0.00016921601029906697
위치: (3, 39), 값: 0.00034516741110479643
위치: (6, 45), 값: 0.00020781786849255468
위치: (7, 46), 값: 0.0003043227036059193
위치: (16, 36), 값: 0.0003732245583947247
위치: (18, 44), 값: 6.87080614980573e-05
위치: (21, 23), 값: 0.0003598787648115831
위치: (28, 48), 값: 0.00033924626980308403
위치: (28, 50), 값: 0.00033796474529879106
위치: (29, 42), 값: 0.00017576188439250018
위치: (29, 43), 값: 0.00024248046339181

In [108]:
import networkx as nx

# 네트워크 생성
G = nx.Graph()

# 인덱스 추가 (위치에 따라)
edges = [
    (0, 35), (1, 34), (3, 16), (3, 39), (6, 45), (7, 46),
    (16, 36), (18, 44), (21, 23), (28, 48), (28, 50),
    (29, 42), (29, 43), (30, 41), (42, 43)
]

# 엣지를 그래프에 추가 (행과 열을 별도의 노드로 처리)
for row, col in edges:
    G.add_node(row)
    G.add_node(col)
    G.add_edge(row, col)

# 연결된 컴포넌트 찾기
connected_components = list(nx.connected_components(G))

# 결과 출력
print("연결된 컴포넌트의 개수:", len(connected_components))
print("각 연결된 컴포넌트:", connected_components)

연결된 컴포넌트의 개수: 10
각 연결된 컴포넌트: [{0, 35}, {1, 34}, {16, 3, 36, 39}, {45, 6}, {46, 7}, {18, 44}, {21, 23}, {48, 50, 28}, {42, 43, 29}, {41, 30}]


In [102]:
lst=[]
for i in range(50):
          if i not in indices[1]:
                  lst.append(i)

In [109]:
import networkx as nx

# 네트워크 생성
G = nx.Graph()

# 인덱스 추가 (위치에 따라)
edges = [
    (0, 35), (1, 34), (3, 16), (3, 39), (6, 45), (7, 46),
    (16, 36), (18, 44), (21, 23), (28, 48), (28, 50),
    (29, 42), (29, 43), (30, 41), (42, 43)
]

# 모든 지역 0~49를 노드로 추가 (50은 포함되지 않았다고 가정)
for i in range(50):
    G.add_node(i)

# 엣지를 그래프에 추가
for row, col in edges:
    G.add_edge(row, col)

# 연결된 컴포넌트 찾기
connected_components = list(nx.connected_components(G))

# 전체 지역 수
total_regions = 50

# 연결된 지역 수 파악
connected_nodes = set()
for component in connected_components:
    connected_nodes.update(component)

# 독립된 지역 수 계산
independent_regions = total_regions - len(connected_nodes)

# 결과 출력
print("독립된 지역의 개수:", independent_regions)
print("연결된 컴포넌트의 개수:", len(connected_components))
print("각 연결된 컴포넌트:", connected_components)


독립된 지역의 개수: -1
연결된 컴포넌트의 개수: 37
각 연결된 컴포넌트: [{0, 35}, {1, 34}, {2}, {16, 3, 36, 39}, {4}, {5}, {45, 6}, {46, 7}, {8}, {9}, {10}, {11}, {12}, {13}, {14}, {15}, {17}, {18, 44}, {19}, {20}, {21, 23}, {22}, {24}, {25}, {26}, {27}, {48, 50, 28}, {42, 43, 29}, {41, 30}, {31}, {32}, {33}, {37}, {38}, {40}, {47}, {49}]


In [76]:
indices

(array([ 0,  1,  2,  3,  3,  3,  5,  6,  7,  7, 16, 16, 18, 21, 28, 28, 29,
        29, 29, 30, 42, 48, 48], dtype=int64),
 array([35, 34, 37, 16, 36, 39, 43, 45, 25, 46, 36, 39, 44, 23, 48, 50, 30,
        42, 43, 41, 43, 49, 50], dtype=int64))

In [75]:
xy

array([[127.01372482,  37.55578326],
       [127.01256006,  37.5565914 ],
       [127.01244746,  37.55597667],
       [127.01159494,  37.55614281],
       [127.01317892,  37.55864876],
       [127.01514913,  37.56032618],
       [127.01266486,  37.55994184],
       [127.01035985,  37.56412539],
       [127.00292074,  37.56323662],
       [126.99375528,  37.56130484],
       [126.98141426,  37.5616652 ],
       [126.9685024 ,  37.56159516],
       [126.97358438,  37.56263706],
       [126.98368685,  37.56063795],
       [126.99833066,  37.56219383],
       [127.01625413,  37.55937801],
       [127.01171345,  37.55619352],
       [127.01283649,  37.55890081],
       [127.0181571 ,  37.56110674],
       [127.01539333,  37.56395095],
       [127.01536497,  37.56476095],
       [127.01254998,  37.56333302],
       [127.01185363,  37.56311605],
       [127.01270995,  37.56353294],
       [127.0117471 ,  37.56367251],
       [127.01083625,  37.56411607],
       [126.9762387 ,  37.56041189],
 

In [58]:
len(indices)

1369

In [97]:
indices[1]

array([35, 34, 37, 16, 36, 39, 43, 45, 25, 46, 36, 39, 44, 23, 48, 50, 30,
       42, 43, 41, 43, 49, 50], dtype=int64)

In [100]:
pd.DataFrame(xy).iloc[[0,1,2,3,5,6,7,16,18,21,23,25,28,29,30,34,35,36,37,39,41,42,43,44,45,46,48,49,50]].to_csv('output.csv', index=True)

In [33]:
B=np.arange(18).reshape(9,2)
B
A.shape

(10, 10)

In [30]:
for i in B:
          for j in B:
                  A[i,j]=i-j

IndexError: index 10 is out of bounds for axis 1 with size 10